In [1]:
import tqdm.notebook as tqdm
import numpy as np
import scipy
import sklearn

# Spark

In [2]:
! /home/jovyan/start-hadoop.sh

jovyan
 * Starting OpenBSD Secure Shell server sshd
start-stop-daemon: unable to set gid to 0 (Operation not permitted)
   ...fail!
 * sshd is running
Starting namenodes on [localhost]
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
localhost: namenode is running as process 160.  Stop it first and ensure /tmp/hadoop-jovyan-namenode.pid file is empty before retry.
Starting datanodes
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
localhost: datanode is running as process 268.  Stop it first and ensure /tmp/hadoop-jovyan-datanode.pid file is empty before retry.
Starting secondary namenodes [bc1b62f69f18]
bc1b62f69f18: Warning: Permanently added 'bc1b62f69f18' (ED25519) to the list of known hosts.
bc1b62f69f18: secondarynamenode is running as process 485.  Stop it first and ensure /tmp/hadoop-jovyan-secondarynamenode.pid file is empty before retry.
Starting resourcemanager
resourcemanager is running as process

In [3]:
# connect, context, session

import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-04-25 07:28:15,126 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
2023-04-25 07:28:21,994 WARN hdfs.DFSOutputStream: NotReplicatedYetException sleeping /user/jovyan/.sparkStaging/application_1682407619857_0001/__spark_libs__6553554960553091851.zip retries left 4


## HDFS

In [4]:
! hdfs dfs -df -h

Filesystem                 Size     Used  Available  Use%
hdfs://localhost:9000  1006.9 G  291.6 M    926.7 G    0%


In [5]:
! hdfs dfs -ls /

Found 2 items
drwxrwx---   - root   supergroup          0 2023-04-25 07:28 /tmp
drwxr-xr-x   - jovyan supergroup          0 2023-04-25 07:28 /user


## Broadcast and accumulator

In [6]:
bc = sc.broadcast({"this": 0, "is": 1, "text": 2})
errors = sc.accumulator(0)

def mapper(x):
    global errors
    for w in x.split():
        if w in bc.value:
            yield (bc.value[w], 1)
        else:
            errors += 1

rdd = (
    sc
   .parallelize(["this is text", "text too"])
   .flatMap(mapper)
   .reduceByKey(lambda a, b: a + b)
)
print(rdd)
print(rdd.collect())
print("errors:", errors.value)

PythonRDD[5] at RDD at PythonRDD.scala:53


[(0, 1), (1, 1), (2, 2)]
errors: 1


## DataFrame API

RDD is much better and useful than plain MapReduce, but Spark can do even more!
Spark DataFrame is table structure over RDDs and can be treated as pandas on steroids.

It allows us to perform structured queries and benefit from it. One way is to perform SQL-styled queries (will discuss on next lesson) and another is DataFrame API.

Documentation: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html

In [7]:
rdd = sc.parallelize([("a", 1), ("a", 2), ("b", 3), ("b", 4)])
rdd.collect()

[('a', 1), ('a', 2), ('b', 3), ('b', 4)]

In [ ]:
df = se.createDataFrame(rdd)
df.printSchema()
df.show()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)



In [9]:
from pyspark.sql import Row

df = se.createDataFrame(
    rdd.map(lambda x: Row(col_one=x[0], col_two=x[1]))
)
df.printSchema()
df.show()

root
 |-- col_one: string (nullable = true)
 |-- col_two: long (nullable = true)



+-------+-------+
|col_one|col_two|
+-------+-------+
|      a|      1|
|      a|      2|
|      b|      3|
|      b|      4|
+-------+-------+



In [10]:
df.select(['col_one']).limit(2).show()

+-------+
|col_one|
+-------+
|      a|
|      a|
+-------+



In [11]:
df.select(['col_one']).distinct().rdd.map(lambda x: x.col_one).collect()

['a', 'b']

Docs: https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#module-pyspark.sql.functions

In [12]:
from pyspark.sql import functions as F

(
  df.select(['col_one', 'col_two'])
    .where(F.col('col_one') == 'a')
    .limit(2)
    .show()
)

+-------+-------+
|col_one|col_two|
+-------+-------+
|      a|      1|
|      a|      2|
+-------+-------+



In [13]:
df = df.select('*', df['col_two'].cast('float').alias('col_two_float'))
df.show()

+-------+-------+-------------+
|col_one|col_two|col_two_float|
+-------+-------+-------------+
|      a|      1|          1.0|
|      a|      2|          2.0|
|      b|      3|          3.0|
|      b|      4|          4.0|
+-------+-------+-------------+



In [14]:
square_df = df.select('col_one', (df['col_two_float'] * df['col_two_float']).alias('col_two_square'))
square_df.orderBy('col_two_square', ascending=False).show(5)

+-------+--------------+
|col_one|col_two_square|
+-------+--------------+
|      b|          16.0|
|      b|           9.0|
|      a|           4.0|
|      a|           1.0|
+-------+--------------+



In [15]:
from pyspark.sql import functions as F

(
    df
      .groupby('col_one')
      .agg(F.collect_list("col_two").alias("col_two_list"))
      .select(['col_one', 'col_two_list'])
      .limit(10)
      .show()
)

+-------+------------+
|col_one|col_two_list|
+-------+------------+
|      a|      [2, 1]|
|      b|      [3, 4]|
+-------+------------+



In [16]:
list(df.toLocalIterator())

[Row(col_one='a', col_two=1, col_two_float=1.0),
 Row(col_one='a', col_two=2, col_two_float=2.0),
 Row(col_one='b', col_two=3, col_two_float=3.0),
 Row(col_one='b', col_two=4, col_two_float=4.0)]

In [17]:
df = se.createDataFrame(df.toLocalIterator())

In [18]:
df.printSchema()
df.show()

root
 |-- col_one: string (nullable = true)
 |-- col_two: long (nullable = true)
 |-- col_two_float: double (nullable = true)



+-------+-------+-------------+
|col_one|col_two|col_two_float|
+-------+-------+-------------+
|      a|      1|          1.0|
|      a|      2|          2.0|
|      b|      3|          3.0|
|      b|      4|          4.0|
+-------+-------+-------------+



In [19]:
# also can get RDD from DF
df.rdd.collect()

[Row(col_one='a', col_two=1, col_two_float=1.0),
 Row(col_one='a', col_two=2, col_two_float=2.0),
 Row(col_one='b', col_two=3, col_two_float=3.0),
 Row(col_one='b', col_two=4, col_two_float=4.0)]

## Data formats

In [20]:
# We may want to operate with not just plain text, but something more complex
# For example, Parquet - it can be useful for huge datasets for faster calcs
df.write.save("data.parquet")

In [21]:
data = se.read.parquet("data.parquet")
data.rdd.collect()

[Row(col_one='a', col_two=1, col_two_float=1.0),
 Row(col_one='a', col_two=2, col_two_float=2.0),
 Row(col_one='b', col_two=3, col_two_float=3.0),
 Row(col_one='b', col_two=4, col_two_float=4.0)]

## Outbrain click prediction dataseet

https://www.kaggle.com/c/outbrain-click-prediction/data - you need to register here

In [ ]:
# https://github.com/Kaggle/kaggle-api
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# instruction where to get ~/.kaggle/kaggle.json file

  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [20]:
kaggle = {"username":"xxxxxxx","key":"xxxxxxx"}

In [21]:
! mkdir -p ~/.kaggle

In [22]:
%%writefile ~/.kaggle/kaggle.json
...

Overwriting /home/jovyan/.kaggle/kaggle.json


In [23]:
import json

with open("/home/jovyan/.kaggle/kaggle.json", "w") as f:
    json.dump(kaggle,f)

In [24]:
! chmod 600 ~/.kaggle/kaggle.json

In [25]:
! pip install -U urllib3 kaggle==1.5.3
! kaggle competitions download -c outbrain-click-prediction -f page_views_sample.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f events.csv.zip 
! kaggle competitions download -c outbrain-click-prediction -f documents_topics.csv.zip 
! kaggle competitions download -c outbrain-click-prediction -f documents_entities.csv.zip 
! kaggle competitions download -c outbrain-click-prediction -f documents_meta.csv.zip 
! kaggle competitions download -c outbrain-click-prediction -f clicks_test.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f clicks_train.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f documents_categories.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f promoted_content.csv.zip

  Using cached urllib3-1.26.15-py2.py3-none-any.whl (140 kB)
page_views_sample.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
events.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
documents_topics.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
100%|████████████████████████████████████████| 126M/126M [00:14<00:00, 9.86MB/s]
100%|████████████████████████████████████████| 126M/126M [00:14<00:00, 9.35MB/s]
 97%|████████████████████████████████████▋ | 15.0M/15.5M [00:02<00:00, 10.0MB/s]
100%|██████████████████████████████████████| 15.5M/15.5M [00:02<00:00, 7.34MB/s]
100%|███████████████████████████████████████▊| 135M/135M [00:14<00:00, 9.81MB/s]
100%|████████████████████████████████████████| 135M/135M [00:14<00:00, 9.48MB/s]
100%|████████████████████████████████████████| 390M/390M [00:46<00:00, 7.24MB/s]
100%|████████████████████████████████████████| 390M/390M 

In [26]:
! unzip '*.zip'
! rm -rf *.zip

Archive:  clicks_test.csv.zip
  inflating: clicks_test.csv         

Archive:  page_views_sample.csv.zip
  inflating: page_views_sample.csv   

Archive:  documents_entities.csv.zip
  inflating: documents_entities.csv  

Archive:  documents_topics.csv.zip
  inflating: documents_topics.csv    

Archive:  documents_categories.csv.zip
  inflating: documents_categories.csv  

Archive:  documents_meta.csv.zip
  inflating: documents_meta.csv      

Archive:  promoted_content.csv.zip
  inflating: promoted_content.csv    

Archive:  clicks_train.csv.zip
  inflating: clicks_train.csv        

Archive:  events.csv.zip
  inflating: events.csv              

9 archives were successfully processed.


In [27]:
! hdfs dfs -put page_views_sample.csv
! hdfs dfs -put events.csv
! hdfs dfs -put documents_topics.csv
! hdfs dfs -put documents_entities.csv
! hdfs dfs -put documents_meta.csv
! hdfs dfs -put clicks_test.csv
! hdfs dfs -put clicks_train.csv
! hdfs dfs -put documents_categories.csv
! hdfs dfs -put promoted_content.csv

In [28]:
# load data
df = se.read.csv("page_views_sample.csv", header=True)
df.show(5)

+--------------+-----------+---------+--------+------------+--------------+
|          uuid|document_id|timestamp|platform|geo_location|traffic_source|
+--------------+-----------+---------+--------+------------+--------------+
|1fd5f051fba643|        120| 31905835|       1|          RS|             2|
|8557aa9004be3b|        120| 32053104|       1|       VN>44|             2|
|c351b277a358f0|        120| 54013023|       1|       KR>12|             1|
|8205775c5387f9|        120| 44196592|       1|       IN>16|             2|
|9cb0ccd8458371|        120| 65817371|       1|   US>CA>807|             2|
+--------------+-----------+---------+--------+------------+--------------+
only showing top 5 rows



### Data manipulations

In [29]:
from IPython.display import display
tables = ["clicks_test", "clicks_train", 
          "documents_categories", "documents_entities", "documents_meta", "documents_topics", 
          "events", "page_views_sample", "promoted_content"]
for name in tqdm.tqdm(tables):
    df = se.read.csv("{}.csv".format(name), header=True)
    df.registerTempTable(name)
    print(name)
    df.limit(3).show()

  0%|          | 0/9 [00:00<?, ?it/s]

clicks_test
+----------+------+
|display_id| ad_id|
+----------+------+
|  16874594| 66758|
|  16874594|150083|
|  16874594|162754|
+----------+------+

clicks_train
+----------+------+-------+
|display_id| ad_id|clicked|
+----------+------+-------+
|         1| 42337|      0|
|         1|139684|      0|
|         1|144739|      1|
+----------+------+-------+

documents_categories
+-----------+-----------+----------------+
|document_id|category_id|confidence_level|
+-----------+-----------+----------------+
|    1595802|       1611|            0.92|
|    1595802|       1610|            0.07|
|    1524246|       1807|            0.92|
+-----------+-----------+----------------+

documents_entities
+-----------+--------------------+-----------------+
|document_id|           entity_id| confidence_level|
+-----------+--------------------+-----------------+
|    1524246|f9eec25663db4cd83...|0.672865314504701|
|    1524246|55ebcfbdaff1d6f60...|0.399113728441297|
|    1524246|839907a972930b17b

In [20]:
page_views = se.table("page_views_sample")
print(page_views)

DataFrame[uuid: string, document_id: string, timestamp: string, platform: string, geo_location: string, traffic_source: string]


In [21]:
page_views.select('*').show()

+--------------+-----------+---------+--------+------------+--------------+
|          uuid|document_id|timestamp|platform|geo_location|traffic_source|
+--------------+-----------+---------+--------+------------+--------------+
|1fd5f051fba643|        120| 31905835|       1|          RS|             2|
|8557aa9004be3b|        120| 32053104|       1|       VN>44|             2|
|c351b277a358f0|        120| 54013023|       1|       KR>12|             1|
|8205775c5387f9|        120| 44196592|       1|       IN>16|             2|
|9cb0ccd8458371|        120| 65817371|       1|   US>CA>807|             2|
|2aa611f32875c7|        120| 71495491|       1|       CA>ON|             2|
|f55a6eaf2b34ab|        120| 73309199|       1|       BR>27|             2|
|cc01b582c8cbff|        120| 50033577|       1|       CA>BC|             2|
|6c802978b8dd4d|        120| 66590306|       1|       CA>ON|             2|
|f4e423314303ff|        120| 48314254|       1|   US>LA>622|             1|
|3937372ca27

In [23]:
page_views_sql = se.sql("SELECT * from page_views_sample")
page_views_sql.show()

+--------------+-----------+---------+--------+------------+--------------+
|          uuid|document_id|timestamp|platform|geo_location|traffic_source|
+--------------+-----------+---------+--------+------------+--------------+
|1fd5f051fba643|        120| 31905835|       1|          RS|             2|
|8557aa9004be3b|        120| 32053104|       1|       VN>44|             2|
|c351b277a358f0|        120| 54013023|       1|       KR>12|             1|
|8205775c5387f9|        120| 44196592|       1|       IN>16|             2|
|9cb0ccd8458371|        120| 65817371|       1|   US>CA>807|             2|
|2aa611f32875c7|        120| 71495491|       1|       CA>ON|             2|
|f55a6eaf2b34ab|        120| 73309199|       1|       BR>27|             2|
|cc01b582c8cbff|        120| 50033577|       1|       CA>BC|             2|
|6c802978b8dd4d|        120| 66590306|       1|       CA>ON|             2|
|f4e423314303ff|        120| 48314254|       1|   US>LA>622|             1|
|3937372ca27

In [24]:
documents_topics = se.table("documents_topics")
print(documents_topics)

DataFrame[document_id: string, topic_id: string, confidence_level: string]


In [29]:
from pyspark.sql import functions as F
from pyspark.sql.functions import desc

(
    page_views
      .join(documents_topics, page_views.document_id == documents_topics.document_id, 'outer')
      .select(page_views.document_id, documents_topics.topic_id)
      .sort(desc("document_id"))
      .limit(50)
      .show()
)

+-----------+--------+
|document_id|topic_id|
+-----------+--------+
|     999924|     184|
|     999924|     184|
|     999924|     223|
|     999924|     223|
|     999924|     223|
|     999924|     223|
|     999924|     184|
|     999924|     223|
|     999924|     184|
|     999924|     223|
|     999924|     184|
|     999924|     223|
|     999924|     184|
|     999924|     184|
|     999924|     223|
|     999924|     184|
|     999924|     184|
|     999924|     184|
|     999924|     223|
|     999924|     184|
+-----------+--------+
only showing top 20 rows



In [30]:
(
  documents_topics
    .groupby('document_id')
    .agg(F.collect_list("topic_id").alias("topics"))
    .limit(10)
    .show()
)

+-----------+--------------------+
|document_id|              topics|
+-----------+--------------------+
|    1000015|          [183, 283]|
|    1000067|[199, 183, 229, 1...|
|    1000073|           [183, 35]|
|    1000096|      [173, 254, 71]|
|     100010|[16, 254, 192, 25...|
|    1000113|      [184, 183, 35]|
|    1000128|           [183, 35]|
|    1000131|[183, 199, 235, 1...|
|    1000146|      [183, 35, 202]|
|    1000167|      [183, 35, 202]|
+-----------+--------------------+



In [31]:
(
  documents_topics
    .groupby('document_id')
    .agg(F.count("topic_id").alias("topics_cnt"))
    .orderBy("topics_cnt")
    .limit(10)
    .show()
)

+-----------+----------+
|document_id|topics_cnt|
+-----------+----------+
|    1451809|         1|
|    2256862|         1|
|    1388193|         1|
|    2284422|         1|
|    1801253|         1|
|    2114730|         1|
|    1877485|         1|
|    2885936|         1|
|     196385|         1|
|      79893|         1|
+-----------+----------+



In [32]:
(
  documents_topics
    .groupby('document_id')
    .agg(F.countDistinct("topic_id").alias("topics_cnt"))
    .orderBy("topics_cnt")
    .limit(10)
    .show()
)

+-----------+----------+
|document_id|topics_cnt|
+-----------+----------+
|    1967394|         1|
|    2044122|         1|
|    2068338|         1|
|    2241850|         1|
|    2362165|         1|
|    2395407|         1|
|    2225431|         1|
|     482213|         1|
|    2458821|         1|
|    2659486|         1|
+-----------+----------+



In [33]:
page_views.select("traffic_source").distinct().show()

+--------------+
|traffic_source|
+--------------+
|             3|
|             1|
|             2|
+--------------+



In [34]:
page_views.select("traffic_source").distinct().explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[traffic_source#445], functions=[])
   +- Exchange hashpartitioning(traffic_source#445, 200), ENSURE_REQUIREMENTS, [id=#819]
      +- HashAggregate(keys=[traffic_source#445], functions=[])
         +- FileScan csv [traffic_source#445] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9000/user/jovyan/page_views_sample.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<traffic_source:string>




In [35]:
from pyspark.sql.window import Window

In [36]:
w = Window.partitionBy('topic_id')\
          .orderBy('confidence_level')
(
  documents_topics
    .withColumn('row_number', F.row_number().over(w) - 1)
    .orderBy("row_number")
    .take(100)
)

[Row(document_id='1373067', topic_id='120', confidence_level='0.00800022845571354', row_number=0),
 Row(document_id='1721729', topic_id='100', confidence_level='0.00800006482315529', row_number=0),
 Row(document_id='16439', topic_id='126', confidence_level='0.0080008012801282', row_number=0),
 Row(document_id='1205392', topic_id='52', confidence_level='0.00800003350319164', row_number=0),
 Row(document_id='113635', topic_id='5', confidence_level='0.00800018262447156', row_number=0),
 Row(document_id='2281646', topic_id='75', confidence_level='0.00800059856285632', row_number=0),
 Row(document_id='1367225', topic_id='140', confidence_level='0.00800012204207135', row_number=0),
 Row(document_id='1450684', topic_id='187', confidence_level='0.00800008391306855', row_number=0),
 Row(document_id='1820377', topic_id='195', confidence_level='0.00800005376000286', row_number=0),
 Row(document_id='1939630', topic_id='179', confidence_level='0.00800007203623664', row_number=0),
 Row(document_id='

## Evaluation Assignment

Data: outbrain click prediction

Tasks:
Using Spark RDD, DataFrame API and Python, calculate:

**1**. Top 10 most visited document_ids in the page_views_sample log

**2**. How many users have at least 2 different traffic_sources in the page_views_sample log (note the value is not a count, it's an encoded enum)

**3***. Top 10 most visited topic_ids in page_views_sample log (use documents_topics table)

The submission format is the result.json json file with top_10_documents, users and top_10_topics keys.
For TOP-10 results, the answer must be written in the form of a sheet ordered from TOP-1 to TOP-10 with an id.

result.json example:

    {
        "top_10_documents": [
            111,
            222,
            333,
            ...,
            1010
        ],
        "users": 10000,
        "top_10_topics": [
            11,
            22,
            33,
            ...,
            101
        ]
    }

### 1. Top 10 most visited document_ids in the page_views_sample log

In [30]:
page_views_sample = se.read.csv("page_views_sample.csv", header=True)
page_views_sample.registerTempTable("page_views_sample")

In [31]:
page_views_sample.limit(5).toPandas()

,uuid,document_id,timestamp,platform,geo_location,traffic_source
0,1fd5f051fba643,120,31905835,1,RS,2
1,8557aa9004be3b,120,32053104,1,VN>44,2
2,c351b277a358f0,120,54013023,1,KR>12,1
3,8205775c5387f9,120,44196592,1,IN>16,2
4,9cb0ccd8458371,120,65817371,1,US>CA>807,2


In [36]:
# first solution

from pyspark.sql import functions as F
from pyspark.sql.functions import desc

(
    page_views_sample
      .groupby('document_id')
      .agg(F.count("document_id").alias("document_cnt"))
      .sort(desc("document_cnt"))
      .limit(10)
      .show()
)

+-----------+------------+
|document_id|document_cnt|
+-----------+------------+
|    1811567|      429551|
|        234|      179692|
|      42744|      156231|
|    1858440|      112140|
|    1780813|      109624|
|      60164|      104941|
|    1790442|       91420|
|    1877626|       80309|
|    1821895|       79956|
|     732651|       74630|
+-----------+------------+



In [37]:
# second solution

top10_docs = se.sql("""
    select document_id, count(*) as cnt 
    from page_views_sample 
    group by document_id 
    order by cnt desc
""").limit(10).toPandas()

In [38]:
top10_docs

,document_id,cnt
0,1811567,429551
1,234,179692
2,42744,156231
3,1858440,112140
4,1780813,109624
5,60164,104941
6,1790442,91420
7,1877626,80309
8,1821895,79956
9,732651,74630


In [70]:
top10_docs_res = []
for i in top10_docs.document_id.tolist():
    top10_docs_res.append(int(i))
print('Top 10 documents:', top10_docs_res)

Top 10 documents: [1811567, 234, 42744, 1858440, 1780813, 60164, 1790442, 1877626, 1821895, 732651]


### 2. How many users have at least 2 different traffic_sources in the page_views_sample log (note the value is not a count, it's an encoded enum)

In [40]:
page_views_sample.limit(5).toPandas()

,uuid,document_id,timestamp,platform,geo_location,traffic_source
0,1fd5f051fba643,120,31905835,1,RS,2
1,8557aa9004be3b,120,32053104,1,VN>44,2
2,c351b277a358f0,120,54013023,1,KR>12,1
3,8205775c5387f9,120,44196592,1,IN>16,2
4,9cb0ccd8458371,120,65817371,1,US>CA>807,2


In [43]:
# first solution

(
  page_views_sample
    .groupby('uuid')
    .agg(F.countDistinct("traffic_source").alias("traffic_cnt"))
    .where("traffic_cnt>=2")
    .count()
)

98080

In [44]:
# second solution

traffic = se.sql("""
    select count(*) as result 
    from
    (select uuid, count(distinct traffic_source) as cnt_traffic 
    from page_views_sample 
    group by uuid 
    having cnt_traffic>=2)
    as count_uuid
""").limit(1).toPandas()

In [45]:
traffic

,result
0,98080


In [59]:
users_traffic_res = traffic.result.tolist()[0]
print('Users with at least 2 different traffic_sources:', users_traffic_res)

Users with at least 2 different traffic_sources: 98080


### 3*. Top 10 most visited topic_ids in page_views_sample log (use documents_topics table)

In [47]:
documents_topics = se.read.csv("documents_topics.csv", header=True)
documents_topics.registerTempTable("documents_topics")

In [48]:
documents_topics.limit(5).toPandas()

,document_id,topic_id,confidence_level
0,1595802,140,0.0731131601068925
1,1595802,16,0.0594164867373976
2,1595802,143,0.0454207537554526
3,1595802,170,0.0388674285182961
4,1524246,113,0.196450402209685


In [78]:
# first solution

from pyspark.sql import functions as F
from pyspark.sql.functions import desc

(
    page_views_sample
      .join(documents_topics, page_views_sample.document_id == documents_topics.document_id)
      .groupby('topic_id')
      .agg(F.count("topic_id").alias("cnt_topic"))
      .select(documents_topics.topic_id)
      .sort(desc("cnt_topic"))
      .limit(10)
      .show()
)

+--------+
|topic_id|
+--------+
|      20|
|      16|
|     216|
|     136|
|     140|
|     143|
|      36|
|      97|
|       8|
|     269|
+--------+



In [49]:
# second solution

top10_topics = se.sql("""
    select topic_id, count(*) as cnt_topic 
    from documents_topics 
    join page_views_sample on documents_topics.document_id=page_views_sample.document_id
    group by topic_id 
    order by cnt_topic desc
""").limit(10).toPandas()

In [50]:
top10_topics

,topic_id,cnt_topic
0,20,1429253
1,16,1425838
2,216,1160563
3,136,1099382
4,140,971983
5,143,919136
6,36,855793
7,97,839328
8,8,819599
9,269,727145


In [71]:
top10_topics_res = []
for i in top10_topics.topic_id.tolist():
    top10_topics_res.append(int(i))
print('Top 10 most visited topics:', top10_topics_res)

Top 10 most visited topics: [20, 16, 216, 136, 140, 143, 36, 97, 8, 269]


In [72]:
import json

final_result = {
    "top_10_documents": top10_docs_res,
    "users": users_traffic_res,
    "top_10_topics": top10_topics_res,
}

In [73]:
with open("result.json", "w") as f:
    json.dump(final_result, f)

In [74]:
with open("result.json", "r") as f:
    result = json.load(f)

In [75]:
result

{'top_10_documents': [1811567,
  234,
  42744,
  1858440,
  1780813,
  60164,
  1790442,
  1877626,
  1821895,
  732651],
 'users': 98080,
 'top_10_topics': [20, 16, 216, 136, 140, 143, 36, 97, 8, 269]}

In [76]:
!curl -F file=@result.json "51.250.54.133:80/MDS-LSML1/valentinaskorina/w4/1"

1.0
Well done!
